# Extended Kalman Filter
This demonstrates the Extended Kalman filter for the discretized pendulum $x'' = -\sin(x)$.  It is an analog on example 7.2 from Sullivan.

In [ ]:
using Plots
using Random
using Distributions
using Statistics
using Printf
using LinearAlgebra
using LaTeXStrings

In [ ]:
default(xtickfont=font(12),  ytickfont=font(12), guidefont=font(12), 
    legendfont=font(10), lw=2,ms=4)

In [ ]:
ω = 1;
Δt = 0.1;
γ = sqrt(.5);
f = X-> [X[1] + Δt * X[2] - Δt^2 * sin(X[1]); X[2]-Δt * sin(X[1])];

F = X-> [1 - Δt^2*cos(X[1]) Δt; -Δt * cos(X[1]) 1];
H = Float64[1 0];
R = [γ^2];

# true initial condition
Xt_0 = Float64[2; 0];

# bad initial condition
X0 = Float64[0;0];
C0 = 10^10 * Float64[1 0; 0 1];

n = 10^2;

# generate data
Xt_path = [copy(Xt_0)];
Random.seed!(500);
y_data = Array{Float64}[];
Xt = copy(Xt_0);
for j in 1:n
    Xt = f(Xt);
    y = H * Xt .+ rand(Normal(0,γ));
    push!(Xt_path, copy(Xt));
    push!(y_data, y);
end


In [ ]:
plot(0:n, [X_[1] for X_ in Xt_path],label=L"$x_n$")
scatter!(1:n, [y_[1] for y_ in y_data], label=L"$y_n$")
xlabel!(L"$n$")

In [ ]:
X_path = [X0];
C_path = [C0];
X = copy(X0);
C = copy(C0);
for i in 1:n
    # predict
    X = f(X);
    C = F(X)*C*F(X)'; 

    # correct
    S = H * C * H' + R;
    K = C * H' * inv(S);
    X = X + K * (y_data[i] - H *X);
    C = C  - K * S * K';

    push!(X_path, copy(X));
    push!(C_path, copy(C));
end

In [ ]:
plot(0:n, [X_[1] for X_ in X_path], ribbon = sqrt.([C_[1,1] for C_ in C_path]),label=L"$\hat{x}_n$")
plot!(0:n, [X_[1] for X_ in Xt_path],label=L"$x_n$")
scatter!(1:n, [y_[1] for y_ in y_data],label=L"$y_n$")
ylims!(-2.5,2.5)
xlabel!(L"$n$")

In [ ]:
plot(0:n, [X_[2] for X_ in X_path], ribbon = sqrt.([C_[2,2] for C_ in C_path]),label=L"$\hat{v}_n$")
plot!(0:n, [X_[2] for X_ in Xt_path], label=L"$v_n$")
ylims!(-2.5,2.5)
xlabel!(L"$n$")